<a href="https://colab.research.google.com/github/beaten-by-the-market/dart_disclosure/blob/main/ollama_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 코랩 환경에서 NVIDA GPU, Ollama 설치 등 필요한 설정

In [ ]:
# Install debconf-utils and preset keyboard configuration
!sudo apt-get -y install debconf-utils
!echo "keyboard-configuration keyboard-configuration/layoutcode string us" | sudo debconf-set-selections
!echo "keyboard-configuration keyboard-configuration/variant select English (US)" | sudo debconf-set-selections

# Run installation commands with DEBIAN_FRONTEND=noninteractive
!curl -fsSL https://ollama.com/install.sh | sh
!DEBIAN_FRONTEND=noninteractive sudo apt update
!DEBIAN_FRONTEND=noninteractive sudo apt install -y pciutils
!DEBIAN_FRONTEND=noninteractive sudo apt-get update && sudo apt-get install -y cuda-drivers

import os
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

# 필요한 라이브러리 설치
!pip install langchain-ollama
!pip install -U langchain-ollama

## 모델 다운로드 및 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p /content/drive/MyDrive/ollama_models
import os
os.environ["OLLAMA_MODELS"] = "/content/drive/MyDrive/ollama_models"

In [ ]:
# ollama 실행
import time
!nohup ollama serve &
time.sleep(5)
!ollama pull bge-m3
!ollama pull exaone3.5:7.8b
!ollama pull bnksys/yanolja-eeve-korean-instruct-10.8b
!ollama pull gemma3:4b
!ollama list

Streaming output truncated to the last 5000 lines.
pulling 83739ee31e69:  57% ▕▏ 6.6 GB/ 11 GB   81 MB/s    1m0spulling manifest 
pulling 83739ee31e69:  57% ▕▏ 6.6 GB/ 11 GB   81 MB/s    1m0spulling manifest 
pulling 83739ee31e69:  57% ▕▏ 6.6 GB/ 11 GB   81 MB/s    1m0spulling manifest 
pulling 83739ee31e69:  57% ▕▏ 6.6 GB/ 11 GB   81 MB/s     59spulling manifest 
pulling 83739ee31e69:  57% ▕▏ 6.6 GB/ 11 GB   81 MB/s     59spulling manifest 
pulling 83739ee31e69:  58% ▕▏ 6.6 GB/ 11 GB   81 MB/s     59spulling manifest 
pulling 83739ee31e69:  58% ▕▏ 6.6 GB/ 11 GB   81 MB/s     59spulling manifest 
pulling 83739ee31e69:  58% ▕▏ 6.6 GB/ 11 GB   81 MB/s     59spulling manifest 
pulling 83739ee31e69:  58% ▕▏ 6.6 GB/ 11 GB   81 MB/s     59spulling manifest 
pulling 83739ee31e69:  58% ▕▏ 6.6 GB/ 11 GB   85 MB/s     56spulling manifest 
pulling 83739ee31e69:  58% ▕▏ 6.7 GB/ 11 GB   85 MB/s     56spulling manifest 
pulling 83739ee31e69:  58% ▕▏ 6.7 GB/ 11 GB   85 MB/s     56spulling manifest 
p

## 필요한 패키지 설치

In [ ]:
!pip install langchain-experimental tiktoken langchain-community pydantic ollama langchain
!pip install -U langchain-ollama

from bs4 import BeautifulSoup
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field, field_validator, ValidationInfo
from typing import Optional, List, Dict, Any
import json
import re

## 함수정의

In [ ]:
# Document 클래스 직접 정의 (langchain_core 미사용 시)
try:
    from langchain_core.documents import Document
except ImportError:
    class Document:
        def __init__(self, page_content, metadata=None):
            self.page_content = page_content
            self.metadata = metadata or {}

# 재무지표 출력 스키마 정의 (단위 필드와 컨텍스트 필드 추가)
class FinancialMetricsOutput(BaseModel):
    EBITDA: Optional[str] = Field(description="EBITDA 값 또는 '-'", default="-")
    EBITDA_unit: Optional[str] = Field(description="EBITDA 단위", default="-")
    EBITDA_context: Optional[str] = Field(description="EBITDA 추출 컨텍스트", default="-")

    Adjusted_EBITDA: Optional[str] = Field(description="조정 EBITDA 값 또는 '-'", default="-")
    Adjusted_EBITDA_unit: Optional[str] = Field(description="조정 EBITDA 단위", default="-")
    Adjusted_EBITDA_context: Optional[str] = Field(description="조정 EBITDA 추출 컨텍스트", default="-")

    Adjusted_Operating_Profit: Optional[str] = Field(description="조정 영업손익 값 또는 '-'", default="-")
    Adjusted_Operating_Profit_unit: Optional[str] = Field(description="조정 영업손익 단위", default="-")
    Adjusted_Operating_Profit_context: Optional[str] = Field(description="조정 영업손익 추출 컨텍스트", default="-")

    Adjusted_Operating_Income: Optional[str] = Field(description="조정 영업이익 값 또는 '-'", default="-")
    Adjusted_Operating_Income_unit: Optional[str] = Field(description="조정 영업이익 단위", default="-")
    Adjusted_Operating_Income_context: Optional[str] = Field(description="조정 영업이익 추출 컨텍스트", default="-")

    CAPEX: Optional[str] = Field(description="CAPEX 값 또는 '-'", default="-")
    CAPEX_unit: Optional[str] = Field(description="CAPEX 단위", default="-")
    CAPEX_context: Optional[str] = Field(description="CAPEX 추출 컨텍스트", default="-")

    FCF: Optional[str] = Field(description="FCF 값 또는 '-'", default="-")
    FCF_unit: Optional[str] = Field(description="FCF 단위", default="-")
    FCF_context: Optional[str] = Field(description="FCF 추출 컨텍스트", default="-")

    Recurring_Profit: Optional[str] = Field(description="경상이익 값 또는 '-'", default="-")
    Recurring_Profit_unit: Optional[str] = Field(description="경상이익 단위", default="-")
    Recurring_Profit_context: Optional[str] = Field(description="경상이익 추출 컨텍스트", default="-")

    EBITDA_Margin: Optional[str] = Field(description="EBITDA 마진 값 또는 '-'", default="-")
    EBITDA_Margin_unit: Optional[str] = Field(description="EBITDA 마진 단위", default="-")
    EBITDA_Margin_context: Optional[str] = Field(description="EBITDA 마진 추출 컨텍스트", default="-")

    ROE: Optional[str] = Field(description="ROE 값 또는 '-'", default="-")
    ROE_unit: Optional[str] = Field(description="ROE 단위", default="-")
    ROE_context: Optional[str] = Field(description="ROE 추출 컨텍스트", default="-")

    EBIT: Optional[str] = Field(description="EBIT 값 또는 '-'", default="-")
    EBIT_unit: Optional[str] = Field(description="EBIT 단위", default="-")
    EBIT_context: Optional[str] = Field(description="EBIT 추출 컨텍스트", default="-")

    Consolidated_EBITDA: Optional[str] = Field(description="연결 EBITDA 값 또는 '-'", default="-")
    Consolidated_EBITDA_unit: Optional[str] = Field(description="연결 EBITDA 단위", default="-")
    Consolidated_EBITDA_context: Optional[str] = Field(description="연결 EBITDA 추출 컨텍스트", default="-")

    Separate_EBITDA: Optional[str] = Field(description="개별 EBITDA 값 또는 '-'", default="-")
    Separate_EBITDA_unit: Optional[str] = Field(description="개별 EBITDA 단위", default="-")
    Separate_EBITDA_context: Optional[str] = Field(description="개별 EBITDA 추출 컨텍스트", default="-")

    # 타입 검증 강화
    @field_validator('*')
    @classmethod
    def validate_numeric(cls, v, info: ValidationInfo):
        if info.field_name.endswith('_unit') or info.field_name.endswith('_context'):
            return v
        if isinstance(v, str) and v != "-":
            return re.sub(r'[^\d.]', '', v)
        return v

# 청크별 재무지표 추출을 위한 스키마
class ChunkFinancialMetrics(FinancialMetricsOutput):
    chunk_id: int = Field(description="청크 ID")

# 단위 표준화 처리 함수
def standardize_unit(value, unit):
    if value == "-" or not value:
        return value

    converters = {
        '원': 1,
        '천원': 1000,
        '백만원': 1_000_000,
        '억원': 100_000_000
    }

    return float(value) * converters.get(unit, 1)

# 다국어 단위 처리 함수
def detect_currency_unit(text):
    currency_units = {
        '달러': 'USD',
        '엔': 'JPY',
        '위안': 'CNY'
    }

    for kr, en in currency_units.items():
        if kr in text:
            return en
    return 'KRW'  # 기본 통화 단위

# HTML 소스를 전처리하고 시멘틱 청킹하는 함수
def process_html_for_semantic_chunking(html_content, source_url="", model_embedding=None):
    soup = BeautifulSoup(html_content, 'html.parser')

    # 스크립트, 스타일 등 불필요한 요소 제거
    for element in soup(['script', 'style', 'head', 'title', 'meta', 'iframe']):
        element.extract()

    # 테이블 처리를 개선
    for table in soup.find_all('table'):
        # 테이블 구조를 텍스트로 변환
        table_text = ""
        rows = table.find_all('tr')

        for row in rows:
            # 각 셀(th 또는 td) 내용 가져오기
            cells = row.find_all(['th', 'td'])
            row_text = " | ".join([cell.get_text(strip=True) for cell in cells])
            table_text += row_text + "\n"

        # 원래 테이블을 변환된 텍스트로 대체
        table_wrapper = soup.new_tag('div')
        table_wrapper.string = "\n" + table_text + "\n"
        table.replace_with(table_wrapper)

    # HTML에서 텍스트 추출
    text = soup.get_text(separator=' ', strip=True)

    # 남은 텍스트 정리
    text = re.sub(r'\s+', ' ', text).strip()

    # 간단한 청킹 로직 (OpenAI Embeddings를 사용할 수 없는 경우)
    try:
        # 실제 시멘틱 청킹 시도
        from langchain_experimental.text_splitter import SemanticChunker
        from langchain_ollama import OllamaEmbeddings

        # Ollama 임베딩 모델 초기화
        embeddings = OllamaEmbeddings(model=model_embedding)

        # Document 객체 생성
        document = Document(
            page_content=text,
            metadata={"source": source_url}
        )

        # SemanticChunker 초기화
        text_splitter = SemanticChunker(embeddings)

        # 문서를 의미적 청킹 수행
        chunks = text_splitter.split_documents([document])
    except (ImportError, Exception) as e:
        print(f"시멘틱 청킹을 사용할 수 없어 기본 청킹을 사용합니다: {e}")
        # 기본 청킹 로직 (고정 길이 기반)
        chunk_size = 1000
        chunks = []
        for i in range(0, len(text), chunk_size):
            chunk_text = text[i:i+chunk_size]
            chunks.append(Document(
                page_content=chunk_text,
                metadata={"source": source_url, "chunk_id": i//chunk_size}
            ))

    return chunks

# 청크별 재무지표 추출 프롬프트 템플릿 수정 (단위 추출 및 컨텍스트 추가)
chunk_template = """
당신은 한국 상장사 재무분석 전문가입니다. 아래 텍스트에서 요청된 재무지표를 추출해주세요.

---
#### 분석할 텍스트 (청크 {chunk_id}):
{content}

---
#### 작업 지시사항:
주어진 텍스트에서 다음 재무지표 값과 단위를 추출하세요:
- 가장 최근 회계연도(당기)의 값을 우선적으로 추출하세요.
- 연결 기준 값을 우선적으로 선택하고, 없는 경우 별도(개별) 기준 값을 추출하세요.
- 숫자와 단위를 분리하여 추출하세요 (예: '64223억원' → "EBITDA": "64223", "EBITDA_unit": "억원")
- 단위가 명시되지 않은 경우 문맥을 기반으로 적절한 단위를 추론하세요 (ROE, 마진은 보통 '%', 금액은 '억원' 등)
- 값을 찾을 수 없는 경우 "-"로 표시하세요.
- 숫자 값이 "-"인 경우 단위도 반드시 "-"로 표시하세요.
- 각 지표에 대해 추출된 부분의 전후 문맥을 20자씩 함께 제공하세요.

---
#### 출력 형식:
JSON 형식으로 응답하세요:
{{
  "chunk_id": {chunk_id},
  "EBITDA": "추출한 값 또는 '-'",
  "EBITDA_unit": "추출한 단위 또는 '-'",
  "EBITDA_context": "추출된 텍스트 전후 문맥",
  "Adjusted_EBITDA": "추출한 값 또는 '-'",
  "Adjusted_EBITDA_unit": "추출한 단위 또는 '-'",
  "Adjusted_EBITDA_context": "추출된 텍스트 전후 문맥",
  "Adjusted_Operating_Profit": "추출한 값 또는 '-'",
  "Adjusted_Operating_Profit_unit": "추출한 단위 또는 '-'",
  "Adjusted_Operating_Profit_context": "추출된 텍스트 전후 문맥",
  "Adjusted_Operating_Income": "추출한 값 또는 '-'",
  "Adjusted_Operating_Income_unit": "추출한 단위 또는 '-'",
  "Adjusted_Operating_Income_context": "추출된 텍스트 전후 문맥",
  "CAPEX": "추출한 값 또는 '-'",
  "CAPEX_unit": "추출한 단위 또는 '-'",
  "CAPEX_context": "추출된 텍스트 전후 문맥",
  "FCF": "추출한 값 또는 '-'",
  "FCF_unit": "추출한 단위 또는 '-'",
  "FCF_context": "추출된 텍스트 전후 문맥",
  "Recurring_Profit": "추출한 값 또는 '-'",
  "Recurring_Profit_unit": "추출한 단위 또는 '-'",
  "Recurring_Profit_context": "추출된 텍스트 전후 문맥",
  "EBITDA_Margin": "추출한 값 또는 '-'",
  "EBITDA_Margin_unit": "추출한 단위 또는 '-'",
  "EBITDA_Margin_context": "추출된 텍스트 전후 문맥",
  "ROE": "추출한 값 또는 '-'",
  "ROE_unit": "추출한 단위 또는 '-'",
  "ROE_context": "추출된 텍스트 전후 문맥",
  "EBIT": "추출한 값 또는 '-'",
  "EBIT_unit": "추출한 단위 또는 '-'",
  "EBIT_context": "추출된 텍스트 전후 문맥",
  "Consolidated_EBITDA": "추출한 값 또는 '-'",
  "Consolidated_EBITDA_unit": "추출한 단위 또는 '-'",
  "Consolidated_EBITDA_context": "추출된 텍스트 전후 문맥",
  "Separate_EBITDA": "추출한 값 또는 '-'",
  "Separate_EBITDA_unit": "추출한 단위 또는 '-'",
  "Separate_EBITDA_context": "추출된 텍스트 전후 문맥"
}}
"""

# 정규표현식 패턴을 개선한 재무지표 추출 함수 (단위 추출 및 컨텍스트 추가)
def extract_financial_metrics_from_chunk(chunk):
    text = chunk.page_content
    metrics = {
        "chunk_id": chunk.metadata.get("chunk_id", 0),
        "EBITDA": "-",
        "EBITDA_unit": "-",
        "EBITDA_context": "-",
        "Adjusted_EBITDA": "-",
        "Adjusted_EBITDA_unit": "-",
        "Adjusted_EBITDA_context": "-",
        "Adjusted_Operating_Profit": "-",
        "Adjusted_Operating_Profit_unit": "-",
        "Adjusted_Operating_Profit_context": "-",
        "Adjusted_Operating_Income": "-",
        "Adjusted_Operating_Income_unit": "-",
        "Adjusted_Operating_Income_context": "-",
        "CAPEX": "-",
        "CAPEX_unit": "-",
        "CAPEX_context": "-",
        "FCF": "-",
        "FCF_unit": "-",
        "FCF_context": "-",
        "Recurring_Profit": "-",
        "Recurring_Profit_unit": "-",
        "Recurring_Profit_context": "-",
        "EBITDA_Margin": "-",
        "EBITDA_Margin_unit": "-",
        "EBITDA_Margin_context": "-",
        "ROE": "-",
        "ROE_unit": "-",
        "ROE_context": "-",
        "EBIT": "-",
        "EBIT_unit": "-",
        "EBIT_context": "-",
        "Consolidated_EBITDA": "-",
        "Consolidated_EBITDA_unit": "-",
        "Consolidated_EBITDA_context": "-",
        "Separate_EBITDA": "-",
        "Separate_EBITDA_unit": "-",
        "Separate_EBITDA_context": "-"
    }

    # 컨텍스트 추출 도우미 함수
    def extract_context(match, text):
        start_pos = match.start()
        end_pos = match.end()
        pre_context = text[max(0, start_pos-20):start_pos]
        post_context = text[end_pos:min(end_pos+20, len(text))]
        return f"{pre_context}[{match.group(0)}]{post_context}"

    # EBITDA 추출
    if m := re.search(r'EBITDA[는|은|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["EBITDA"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["EBITDA_unit"] = m.group(2)
        metrics["EBITDA_context"] = extract_context(m, text)

    # 연결 EBITDA 추출
    if m := re.search(r'연결\s*EBITDA[는|은|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["Consolidated_EBITDA"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["Consolidated_EBITDA_unit"] = m.group(2)
        metrics["Consolidated_EBITDA_context"] = extract_context(m, text)

    # 개별 EBITDA 추출
    if m := re.search(r'개별\s*EBITDA[는|은|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["Separate_EBITDA"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["Separate_EBITDA_unit"] = m.group(2)
        metrics["Separate_EBITDA_context"] = extract_context(m, text)

    # 조정 EBITDA 추출
    if m := re.search(r'조정\s*EBITDA[는|은|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["Adjusted_EBITDA"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["Adjusted_EBITDA_unit"] = m.group(2)
        metrics["Adjusted_EBITDA_context"] = extract_context(m, text)

    # CAPEX 추출
    if m := re.search(r'CAPEX[는|은|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["CAPEX"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["CAPEX_unit"] = m.group(2)
        metrics["CAPEX_context"] = extract_context(m, text)

    # FCF 추출
    if m := re.search(r'FCF[는|은|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["FCF"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["FCF_unit"] = m.group(2)
        metrics["FCF_context"] = extract_context(m, text)

    # 잉여 현금 흐름 추출 (FCF)
    if m := re.search(r'잉여\s*현금\s*흐름[은|는|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["FCF"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["FCF_unit"] = m.group(2)
        metrics["FCF_context"] = extract_context(m, text)

    # ROE 추출
    if m := re.search(r'ROE[는|은|이|가|:|=|\s]*([\d\.]+)(%|퍼센트)?', text):
        metrics["ROE"] = m.group(1)
        if m.group(2):
            metrics["ROE_unit"] = "%"
        metrics["ROE_context"] = extract_context(m, text)

    # EBIT 추출
    if m := re.search(r'EBIT[는|은|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["EBIT"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["EBIT_unit"] = m.group(2)
        metrics["EBIT_context"] = extract_context(m, text)

    # EBITDA 마진 추출
    if m := re.search(r'EBITDA\s*마진[은|는|이|가|:|=|\s]*([\d\.]+)(%|퍼센트)?', text):
        metrics["EBITDA_Margin"] = m.group(1)
        if m.group(2):
            metrics["EBITDA_Margin_unit"] = "%"
        metrics["EBITDA_Margin_context"] = extract_context(m, text)

    # 조정 영업이익 추출
    if m := re.search(r'조정\s*영업이익[은|는|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["Adjusted_Operating_Income"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["Adjusted_Operating_Income_unit"] = m.group(2)
        metrics["Adjusted_Operating_Income_context"] = extract_context(m, text)

    # 조정 영업손익 추출
    if m := re.search(r'조정\s*영업손익[은|는|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["Adjusted_Operating_Profit"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["Adjusted_Operating_Profit_unit"] = m.group(2)
        metrics["Adjusted_Operating_Profit_context"] = extract_context(m, text)

    # 경상이익 추출
    if m := re.search(r'경상이익[은|는|이|가|:|=|\s]*([\d,\.]+)([조|억|백만|천|만]?[원|달러|USD|KRW])?', text):
        metrics["Recurring_Profit"] = m.group(1).replace(",", "")
        if m.group(2):
            metrics["Recurring_Profit_unit"] = m.group(2)
        metrics["Recurring_Profit_context"] = extract_context(m, text)

    return metrics

# HTML 소스에서 재무 지표를 추출하는 통합 파이프라인
def process_financial_html(html_content, api_key=None, model_llm=None, model_embedding=None):
    """HTML 소스에서 시멘틱 청킹 후 각 청크별로 재무 지표 추출"""

    # HTML 전처리 및 시멘틱 청킹
    print("HTML 소스 전처리 및 시멘틱 청킹 중...")
    chunks = process_html_for_semantic_chunking(html_content, model_embedding=model_embedding)
    print(f"총 {len(chunks)}개의 청크로 분할되었습니다.")

    # 결과 저장 딕셔너리
    all_metrics = []

    try:
        # Ollama LLM 모델 설정
        model = ChatOllama(
            model=model_llm,
            temperature=0
        )

        # 청크별 출력 파서 생성
        chunk_output_parser = JsonOutputParser(pydantic_object=ChunkFinancialMetrics)

        # 청크별 프롬프트 템플릿 생성
        chunk_prompt_template = PromptTemplate.from_template(chunk_template)

        # 각 청크별로 정보 추출
        print("각 청크별 재무 지표 추출 중...")
        for i, chunk in enumerate(chunks):
            chunk_id = chunk.metadata.get("chunk_id", i)
            print(f"청크 {chunk_id+1}/{len(chunks)} 처리 중...")

            # 청크 내용 미리보기 출력 (최대 200자)
            chunk_preview = chunk.page_content[:200] if len(chunk.page_content) > 200 else chunk.page_content
            print(f"청크 내용 미리보기: {chunk_preview}")

            # 프롬프트, 모델, 출력 파서를 연결
            chain = chunk_prompt_template | model | chunk_output_parser

            try:
                # LLM으로 재무 지표 추출 시도
                chunk_result = chain.invoke({
                    "chunk_id": chunk_id,
                    "content": chunk.page_content
                })
                all_metrics.append(chunk_result)
            except Exception as e:
                print(f"LLM 추출 실패, 규칙 기반 추출로 대체: {e}")
                # LLM 추출 실패 시 규칙 기반 추출 사용
                fallback_result = extract_financial_metrics_from_chunk(chunk)
                all_metrics.append(fallback_result)

        # 청크별 결과 병합 (단위 및 컨텍스트 처리 추가)
        print("청크별 결과 병합 중...")
        # 필드 그룹 분류
        metric_keys = [field for field in FinancialMetricsOutput.model_fields.keys()
                      if not field.endswith('_unit') and not field.endswith('_context')]
        unit_keys = [field for field in FinancialMetricsOutput.model_fields.keys()
                    if field.endswith('_unit')]
        context_keys = [field for field in FinancialMetricsOutput.model_fields.keys()
                       if field.endswith('_context')]

        # 기본값 설정
        merged_metrics = {key: "-" for key in metric_keys}
        merged_units = {key: "-" for key in unit_keys}
        merged_contexts = {key: "-" for key in context_keys}

        # 모든 청크의 결과를 순회하며 값이 있는 첫 번째 항목 선택
        for key in metric_keys:
            for chunk_metrics in all_metrics:
                if key in chunk_metrics and chunk_metrics[key] != "-":
                    merged_metrics[key] = chunk_metrics[key]

                    # 해당 값의 단위도 함께 병합
                    unit_key = f"{key}_unit"
                    if unit_key in chunk_metrics and chunk_metrics.get(unit_key, "-") != "-":
                        merged_units[unit_key] = chunk_metrics[unit_key]

                    # 해당 값의 컨텍스트도 함께 병합
                    context_key = f"{key}_context"
                    if context_key in chunk_metrics and chunk_metrics.get(context_key, "-") != "-":
                        merged_contexts[context_key] = chunk_metrics[context_key]

                    break

        # 단위와 컨텍스트 정보를 병합된 지표에 추가
        merged_output = {}
        merged_output.update(merged_metrics)
        merged_output.update(merged_units)
        merged_output.update(merged_contexts)

    except Exception as e:
        print(f"재무 지표 추출 중 오류 발생: {e}")
        # 규칙 기반 추출로 대체
        print("규칙 기반 추출 사용 중...")
        all_rule_metrics = [extract_financial_metrics_from_chunk(chunk) for chunk in chunks]

        # 청크별 결과 병합
        metric_keys = [field for field in FinancialMetricsOutput.model_fields.keys()
                      if not field.endswith('_unit') and not field.endswith('_context')]
        unit_keys = [field for field in FinancialMetricsOutput.model_fields.keys()
                    if field.endswith('_unit')]
        context_keys = [field for field in FinancialMetricsOutput.model_fields.keys()
                       if field.endswith('_context')]

        # 기본값 설정
        merged_metrics = {key: "-" for key in metric_keys}
        merged_units = {key: "-" for key in unit_keys}
        merged_contexts = {key: "-" for key in context_keys}

        # 모든 청크의 결과를 순회하며 값이 있는 첫 번째 항목 선택
        for key in metric_keys:
            for chunk_metrics in all_rule_metrics:
                if key in chunk_metrics and chunk_metrics[key] != "-":
                    merged_metrics[key] = chunk_metrics[key]

                    # 해당 값의 단위도 함께 병합
                    unit_key = f"{key}_unit"
                    if unit_key in chunk_metrics and chunk_metrics.get(unit_key, "-") != "-":
                        merged_units[unit_key] = chunk_metrics[unit_key]

                    # 해당 값의 컨텍스트도 함께 병합
                    context_key = f"{key}_context"
                    if context_key in chunk_metrics and chunk_metrics.get(context_key, "-") != "-":
                        merged_contexts[context_key] = chunk_metrics[context_key]

                    break

        # 단위와 컨텍스트 정보를 병합된 지표에 추가
        merged_output = {}
        merged_output.update(merged_metrics)
        merged_output.update(merged_units)
        merged_output.update(merged_contexts)

    return merged_output


## 예시본문 불러오기(포스코스틸리온)

In [ ]:
# 불러오기 경로 설정
file_path = '/content/drive/MyDrive/rag_projects/mda_example.txt'

# 파일 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    html_source = f.read()

# 결과 확인
print(html_source[:500])  # 앞 500자만 미리 보기

### 구글 Gemma3

In [ ]:
model_embedding = 'bge-m3'
model_llm = 'gemma3:4b'

# 함수실행
if __name__ == "__main__":
    # 처리 실행
    result = process_financial_html(html_source, model_llm=model_llm, model_embedding=model_embedding)

    # 결과 출력
    print(json.dumps(result, ensure_ascii=False, indent=2))

### LG 엑사원

In [ ]:
model_embedding = 'bge-m3'
model_llm = 'exaone3.5:7.8b'

# 함수실행
if __name__ == "__main__":
    # 처리 실행
    result = process_financial_html(html_source, model_llm=model_llm, model_embedding=model_embedding)

    # 결과 출력
    print(json.dumps(result, ensure_ascii=False, indent=2))

### 야놀자 이브이

In [ ]:
model_embedding = 'bge-m3'
model_llm = 'bnksys/yanolja-eeve-korean-instruct-10.8b'

# 함수실행
if __name__ == "__main__":
    # 처리 실행
    result = process_financial_html(html_source, model_llm=model_llm, model_embedding=model_embedding)

    # 결과 출력
    print(json.dumps(result, ensure_ascii=False, indent=2))

## 예시본문 불러오기(크래프톤)

In [ ]:
# 불러오기 경로 설정
file_path = '/content/drive/MyDrive/rag_projects/krafton_mda.txt'

# 파일 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    html_source = f.read()

# 결과 확인
print(html_source[:500])  # 앞 500자만 미리 보기

### 구글 Gemma3

In [ ]:
model_embedding = 'bge-m3'
model_llm = 'gemma3:4b'

# 함수실행
if __name__ == "__main__":
    # 처리 실행
    result = process_financial_html(html_source, model_llm=model_llm, model_embedding=model_embedding)

    # 결과 출력
    print(json.dumps(result, ensure_ascii=False, indent=2))

### LG 엑사원

In [ ]:
model_embedding = 'bge-m3'
model_llm = 'exaone3.5:7.8b'

# 함수실행
if __name__ == "__main__":
    # 처리 실행
    result = process_financial_html(html_source, model_llm=model_llm, model_embedding=model_embedding)

    # 결과 출력
    print(json.dumps(result, ensure_ascii=False, indent=2))

### 야놀자 이브이

In [ ]:
model_embedding = 'bge-m3'
model_llm = 'bnksys/yanolja-eeve-korean-instruct-10.8b'

# 함수실행
if __name__ == "__main__":
    # 처리 실행
    result = process_financial_html(html_source, model_llm=model_llm, model_embedding=model_embedding)

    # 결과 출력
    print(json.dumps(result, ensure_ascii=False, indent=2))